In [1]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from sklearn.metrics import roc_auc_score
from deepctr.models import DeepFM,DIN
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
import random
import time
import pandas
import keras
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
def load_variable(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

### start training(use 4/1-5/3 model train5/4 data)¶

In [32]:
df_uid_5 = load_variable('/home/jovyan/dataset/may_df_uid')
df_ven_guid_5 = load_variable('/home/jovyan/dataset/may_df_ven_guid')
df_uid54 = df_uid_5[df_uid_5.month==54]
df_ven_guid54 = df_ven_guid_5[df_ven_guid_5.month==54]

In [40]:
df_temp = pd.concat([df_uid54[['month','gid_encode']],df_ven_guid54[['month','gid_encode']]],ignore_index=True)
under = 0
freq_list = []
freq = df_temp.gid_encode.value_counts()
for i in freq.values:
    under += np.log10(i+1)
for i in freq.values:
    freq_list.append(np.log10(i+1)/under)

In [42]:
def create_train_set(df,bywhich):
    t_set = []
    neg_list = random.choices(freq.index,weights = freq_list, k=1000000)
    for userid, hist in df.groupby(bywhich):
        pos_list = hist['gid_encode'].tolist()
        if len(pos_list) > 101:
            pos_list = pos_list[-101:]
        def gen_neg():
            count = 0
            neg = pos_list[0]
            while neg in pos_list:
                if count == 100:
                    count = 0
                neg = random.choices(neg_list, k=1)[0]
                count += 1
            return neg
        neg_list2 = [gen_neg() for i in range(len(pos_list))]
        for i in range(1, len(pos_list)):
            hist = pos_list[:i] 
            t_set.append((userid, hist, pos_list[i], 1))
            t_set.append((userid, hist, neg_list2[i], 0))
    return t_set
uid_train_set = create_train_set(df_uid54,'uid')
ven_guid_train_set = create_train_set(df_ven_guid54,'ven_guid')

In [43]:
train_set = uid_train_set+ven_guid_train_set
np.random.shuffle(train_set)
len(uid_train_set),len(ven_guid_train_set),len(train_set)

(186026, 207944, 393970)

In [44]:
def create_train_array(data):
    g_id = []
    hist_g_id = []
    click = []
    seq_length = []

    for index, value in enumerate(data):
        hist_g_id.append(value[1])
        seq_length.append(len(value[1]))
        g_id.append(value[2])
        click.append(value[3])  
    for i in range(len(hist_g_id)):
        if len(hist_g_id[i]) != 100:
            hist_g_id[i] += (100-len(hist_g_id[i]))*[0]

    g_id = np.array(g_id)
    hist_g_id = np.array(hist_g_id)
    click = np.array(click)
    seq_length = np.array(seq_length)
    return g_id, hist_g_id, click, seq_length
g_id_train, hist_g_id_train, click_train, seq_length_train = create_train_array(train_set)

In [154]:
behavior_feature_list = ["g_id"]
feature_columns = [SparseFeat('g_id', 1870599, embedding_dim=8)]
feature_columns += [VarLenSparseFeat(SparseFeat('hist_g_id',vocabulary_size=1870599, embedding_dim=8, embedding_name='g_id'), maxlen = 100, length_name='seq_le
                                     
x_train = {'g_id': g_id_train, 
        'hist_g_id': hist_g_id_train, 
        'seq_length': seq_length_train}
y_train = click_train

In [157]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/checkpoint_folder/model_401_503")
model.load_weights(latest)

checkpoint_path = "/home/jovyan/dataset/checkpoint_folder/model_401_504/rakuten_0401_0504.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
history401_504 = model.fit(x_train, y_train, batch_size=128, verbose=1, epochs=10, callbacks=[cp_callback])

Epoch 1/10
3078/3078 [==============================] - 718s 228ms/step - loss: 0.5538 - accuracy: 0.7130 - auc: 0.7877

Epoch 00001: saving model to /home/jovyan/dataset/checkpoint_folder/model_401_504/rakuten_0401_0504.ckpt
Epoch 2/10
3078/3078 [==============================] - 763s 248ms/step - loss: 0.4805 - accuracy: 0.7691 - auc: 0.8479

Epoch 00002: saving model to /home/jovyan/dataset/checkpoint_folder/model_401_504/rakuten_0401_0504.ckpt
Epoch 3/10
3078/3078 [==============================] - 816s 265ms/step - loss: 0.4118 - accuracy: 0.8152 - auc: 0.8915

Epoch 00003: saving model to /home/jovyan/dataset/checkpoint_folder/model_401_504/rakuten_0401_0504.ckpt
Epoch 4/10
3078/3078 [==============================] - 814s 264ms/step - loss: 0.3488 - accuracy: 0.8531 - auc: 0.9231

Epoch 00004: saving model to /home/jovyan/dataset/checkpoint_folder/model_401_504/rakuten_0401_0504.ckpt
Epoch 5/10
3078/3078 [==============================] - 777s 252ms/step - loss: 0.2964 - accurac

### 15個epoch

In [46]:
behavior_feature_list = ["g_id"]
feature_columns = [SparseFeat('g_id', 1870599, embedding_dim=8)]
feature_columns += [VarLenSparseFeat(SparseFeat('hist_g_id',vocabulary_size=1870599, embedding_dim=8, embedding_name='g_id'), maxlen = 100, length_name='seq_length')]
x = {'g_id': g_id_train, 
        'hist_g_id': hist_g_id_train, 
        'seq_length': seq_length_train}
y = click_train

In [52]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/checkpoint_folder/model_401_504")
model.load_weights(latest)

checkpoint_path = "/home/jovyan/dataset/15epoch/401_504.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
histtory15epoch = model.fit(x, y, batch_size=128, verbose=1, epochs=5, callbacks=[cp_callback])

Epoch 1/5
3078/3078 [==============================] - 509s 164ms/step - loss: 0.3583 - accuracy: 0.8534 - auc: 0.9213

Epoch 00001: saving model to /home/jovyan/dataset/15epoch/401_504.ckpt
Epoch 2/5
3078/3078 [==============================] - 525s 171ms/step - loss: 0.2648 - accuracy: 0.8993 - auc: 0.9572

Epoch 00002: saving model to /home/jovyan/dataset/15epoch/401_504.ckpt
Epoch 3/5
3078/3078 [==============================] - 530s 172ms/step - loss: 0.2113 - accuracy: 0.9241 - auc: 0.9726

Epoch 00003: saving model to /home/jovyan/dataset/15epoch/401_504.ckpt
Epoch 4/5
3078/3078 [==============================] - 526s 171ms/step - loss: 0.1799 - accuracy: 0.9383 - auc: 0.9799

Epoch 00004: saving model to /home/jovyan/dataset/15epoch/401_504.ckpt
Epoch 5/5
3078/3078 [==============================] - 530s 172ms/step - loss: 0.1595 - accuracy: 0.9473 - auc: 0.9838

Epoch 00005: saving model to /home/jovyan/dataset/15epoch/401_504.ckpt


### 20 epoch

In [54]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/15epoch/model401_504")
model.load_weights(latest)
checkpoint_path = "/home/jovyan/dataset/20epoch/model401_504/401_504.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
histtory5epoch = model.fit(x_train, y_train, batch_size=128, verbose=1, epochs=5, callbacks=[cp_callback])

Epoch 1/5
3078/3078 [==============================] - 865s 280ms/step - loss: 0.1425 - accuracy: 0.9540 - auc: 0.9869

Epoch 00001: saving model to /home/jovyan/dataset/20epoch/model401_504/401_504.ckpt
Epoch 2/5
3078/3078 [==============================] - 843s 274ms/step - loss: 0.1255 - accuracy: 0.9609 - auc: 0.9895

Epoch 00002: saving model to /home/jovyan/dataset/20epoch/model401_504/401_504.ckpt
Epoch 3/5
3078/3078 [==============================] - 844s 274ms/step - loss: 0.1154 - accuracy: 0.9651 - auc: 0.9908

Epoch 00003: saving model to /home/jovyan/dataset/20epoch/model401_504/401_504.ckpt
Epoch 4/5
3078/3078 [==============================] - 845s 275ms/step - loss: 0.1039 - accuracy: 0.9692 - auc: 0.9923

Epoch 00004: saving model to /home/jovyan/dataset/20epoch/model401_504/401_504.ckpt
Epoch 5/5
3078/3078 [==============================] - 776s 252ms/step - loss: 0.0993 - accuracy: 0.9713 - auc: 0.9928

Epoch 00005: saving model to /home/jovyan/dataset/20epoch/model4